In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()


In [5]:
# To give you a preview -- calling OpenAI with these messages is this easy.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


## OK onwards with our first project

In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
# Let's try one out. Change the website and add print statements to follow along.

wb = Website("https://markets.businessinsider.com/")
print(wb.title)
print(wb.text)

Markets Insider: Stock Market News, Realtime Quotes and Charts

Business Insider
My Markets Watchlist
Markets
My Markets Watchlist
Stocks
Indices
Commodities
Cryptocurrencies
Currencies
ETFs
News
Calendar icon
An icon in the shape of a calendar.
Search
Business
Strategy
Economy
Finance
Retail
Advertising
Careers
Media
Real Estate
Small Business
The Better Work Project
Tech
Science
AI
Enterprise
Transportation
Startups
Innovation
Markets
Stocks
Indices
Commodities
Crypto
Currencies
ETFs
Lifestyle
Entertainment
Culture
Travel
Food
Health
Parenting
Reviews
Tech
Streaming
Tickets
Home
Kitchen
Style
Beauty
Gifts
Deals
Politics
Military & Defense
Law
Education
Personal Finance
Banking
Savings
Best Savings Accounts
Raisin Review
Checking Accounts
Best Checking Accounts
Chime Checking Account
SoFi Checking and Saving
Best Money Market Accounts
CDs
Best CD Rates
Best Bank Account Bonuses
Cash Back Debit Cards
Best Banks
CIT Bank Review
UFB Direct Bank
Small Business Banking
Credit Cards
Credit 

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
print(user_prompt_for(wb))

You are looking at a website titled Markets Insider: Stock Market News, Realtime Quotes and Charts

The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Business Insider
My Markets Watchlist
Markets
My Markets Watchlist
Stocks
Indices
Commodities
Cryptocurrencies
Currencies
ETFs
News
Calendar icon
An icon in the shape of a calendar.
Search
Business
Strategy
Economy
Finance
Retail
Advertising
Careers
Media
Real Estate
Small Business
The Better Work Project
Tech
Science
AI
Enterprise
Transportation
Startups
Innovation
Markets
Stocks
Indices
Commodities
Crypto
Currencies
ETFs
Lifestyle
Entertainment
Culture
Travel
Food
Health
Parenting
Reviews
Tech
Streaming
Tickets
Home
Kitchen
Style
Beauty
Gifts
Deals
Politics
Military & Defense
Law
Education
Personal Finance
Banking
Savings
Best Savings Accounts
Raisin Review
Checking Accounts
Best Checking Accounts
Chime Checking Account

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [11]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "Why i am  single ?"}
]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Ah, the classic dilemma! Maybe you're just too fabulous to settle for anything less than perfection. Or it could be that you've mastered the art of enjoying your own company, which is honestly a rare superpower. Or, let's face it—sometimes finding someone who can keep up with your awesomeness is just a tall order. Either way, keep shining! You’re doing great, and the right person is out there, probably just as perplexed about their own singleness.


## And now let's build useful messages for GPT-4o-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
# Try this out, and then try for a few more websites

messages_for(wb)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Markets Insider: Stock Market News, Realtime Quotes and Charts\n\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nBusiness Insider\nMy Markets Watchlist\nMarkets\nMy Markets Watchlist\nStocks\nIndices\nCommodities\nCryptocurrencies\nCurrencies\nETFs\nNews\nCalendar icon\nAn icon in the shape of a calendar.\nSearch\nBusiness\nStrategy\nEconomy\nFinance\nRetail\nAdvertising\nCareers\nMedia\nReal Estate\nSmall Business\nThe Better Work Project\nTech\nScience\nAI\nEnterprise\nTransportation\nStartups\nInnovation\nMarkets\nStocks\nIndices\nCommodities\nCrypto\nCurrencies\nETFs\nLifestyle\nEntertainment\nCultur

## Time to bring it together - the API for OpenAI is very simple!

In [16]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [17]:
summarize("https://markets.businessinsider.com/")

'# Markets Insider Summary\n\nMarkets Insider serves as a comprehensive platform for stock market news, real-time quotes, and financial analysis. The site covers a wide range of financial instruments including stocks, indices, commodities, cryptocurrencies, and ETFs. It provides users with insights on market trends, news about various sectors, and economic forecasts.\n\n### Recent News Highlights:\n1. **Opendoor Revival:** Retail traders have rallied around Opendoor, previously at risk of delisting, fueled by the interest of a bullish hedge fund manager.\n2. **Trump and the Fed:** Concerns have been raised by Wall Street experts regarding the possibility of Trump dismissing Fed Chair Jerome Powell, emphasizing the importance of Fed independence.\n3. **Oil Price Concerns:** A prominent investor warned that falling oil prices might soon become unprofitable for producers, due to continued price slumps throughout the year.\n4. **60/40 Portfolio Performance:** Morningstar indicates that the

In [18]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary("https://markets.businessinsider.com/")

# Markets Insider Summary

**Website Overview:**
Markets Insider is a financial news platform that provides real-time market data, stock quotes, commodity values, and comprehensive news coverage on various financial topics including stocks, indices, currencies, and cryptocurrencies. It is a part of Business Insider, focusing specifically on market-related information.

## Key Features:
- **Real-time Market Data:** Up-to-date information on stocks, commodities, currencies, and ETFs.
- **Financial News:** Articles and announcements covering market trends, investment strategies, and economic analysis.
- **Earnings Calendar:** A schedule featuring upcoming earnings reports from various companies.
- **Market Insights:** In-depth analysis and commentary on market performance and investment opportunities.

## Recent News Highlights:
1. **Opendoor Stock Revival:** Retail traders are driving a resurgence in Opendoor’s penny stock, previously facing delisting, due to interests in short squeezes.
   
2. **Concerns Over Federal Reserve:** Financial experts express opposition to the possibility of Trump dismissing Fed Chair Jerome Powell, citing the importance of Fed independence.

3. **Oil Prices at a Critical Low:** Predictions suggest oil prices may drop to levels that threaten profitability for producers by 2025.

4. **Underperformance of the 60/40 Portfolio:** The popular investment strategy is experiencing its worst stretch of underperformance in 150 years, primarily due to bond performance.

5. **Crypto as Mortgage Asset:** There are discussions suggesting that owning cryptocurrency could soon be counted as an asset in mortgage risk assessments.

6. **Labor Market Warning Signs:** Economists highlight potential weaknesses in the labor market, citing reduced hiring intentions among small businesses.

7. **Investment Sentiment:** There’s notable concern among investors regarding high stock valuations, raising alarms about potential market corrections.

## Future Earnings Announcements:
- **Reliance Industries, American Express, and 3M** are among the companies expected to report their Q2 2025 earnings on July 18, 2025.

**Overall**, Markets Insider serves as a critical hub for investors seeking the latest market updates and financial insights.